In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv(r"C:\Users\miriy\Downloads\sonar_train.data", header = None)
test_data = pd.read_csv(r"C:\Users\miriy\Downloads\sonar_test.data", header= None)
valid_data = pd.read_csv(r"C:\Users\miriy\Downloads\sonar_valid.data", header = None)

In [3]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0392,0.0108,0.0267,0.0257,0.0410,0.0491,0.1053,0.1690,0.2105,0.2471,...,0.0083,0.0080,0.0026,0.0079,0.0042,0.0071,0.0044,0.0022,0.0014,2
1,0.1021,0.0830,0.0577,0.0627,0.0635,0.1328,0.0988,0.1787,0.1199,0.1369,...,0.0709,0.0317,0.0309,0.0252,0.0087,0.0177,0.0214,0.0227,0.0106,2
2,0.0025,0.0309,0.0171,0.0228,0.0434,0.1224,0.1947,0.1661,0.1368,0.1430,...,0.0149,0.0077,0.0036,0.0114,0.0085,0.0101,0.0016,0.0028,0.0014,1
3,0.0335,0.0134,0.0696,0.1180,0.0348,0.1180,0.1948,0.1607,0.3036,0.4372,...,0.0244,0.0232,0.0093,0.0159,0.0193,0.0032,0.0377,0.0126,0.0156,2
4,0.0100,0.0194,0.0155,0.0489,0.0839,0.1009,0.1627,0.2071,0.2696,0.2990,...,0.0130,0.0073,0.0077,0.0075,0.0060,0.0080,0.0019,0.0053,0.0019,1


In [4]:
train_y = train_data.iloc[:,60].values
test_y = test_data.iloc[:,60].values
valid_y = valid_data.iloc[:,60].values

In [5]:
train_y[train_y==2]=-1
test_y[test_y==2]=-1
valid_y[valid_y==2]=-1

In [6]:
train_y = train_y.reshape((train_data.shape[0],1))
test_y = test_y.reshape((test_data.shape[0],1))
valid_y = valid_y.reshape((valid_data.shape[0],1))

In [7]:
train_data = train_data.iloc[:,0:60].values
test_data = test_data.iloc[:,0:60].values
valid_data = valid_data.iloc[:,0:60].values

In [8]:
train_data

array([[0.0392, 0.0108, 0.0267, ..., 0.0044, 0.0022, 0.0014],
       [0.1021, 0.083 , 0.0577, ..., 0.0214, 0.0227, 0.0106],
       [0.0025, 0.0309, 0.0171, ..., 0.0016, 0.0028, 0.0014],
       ...,
       [0.031 , 0.0221, 0.0433, ..., 0.0056, 0.0067, 0.0054],
       [0.0366, 0.0421, 0.0504, ..., 0.0017, 0.0027, 0.0027],
       [0.0117, 0.0069, 0.0279, ..., 0.0062, 0.0026, 0.0052]])

In [9]:
def sigmoid(weight,bias):
    x = np.dot(train_data,weight)+bias
    return (1/(1+np.exp(-x)))

In [10]:
def loss_calc(param,l,weight,bias,data,y):
    t = np.dot(data,weight)+bias
    if param=="l0":
        return np.sum(y*t-np.log(1+np.exp(t)))
    elif param == "l1":
        return np.sum(y*t-np.log(1+np.exp(t)))-l*np.linalg.norm(weight)
    else:
        return np.sum(y*t-np.log(1+np.exp(t))) -l*(np.linalg.norm(weight) ** 2)

In [11]:
def regularization(param,w,l):
    if param=="l0":
        return np.zeros((60,1))
    else:
        return l*w

In [12]:
def logistic_regression(param,l,weight,bias,learning_rate):
    iters = 0
    while True:
        loss= loss_calc(param,l,weight,bias,train_data,train_y)
        iters+=1
        w_grad = np.sum(train_data*((train_y+1)/2-sigmoid(weight,bias)),axis = 0).reshape((60,1))- regularization(param,weight,l)
        b_grad = np.sum((train_y+1)/2-sigmoid(weight,bias),axis=0)
        weight += learning_rate*w_grad 
        bias += learning_rate*b_grad
        loss_new = loss_calc(param,l,weight,bias,train_data,train_y)
        #print(iters,loss,loss_new)
        if iters > 2 and (loss_new-loss)<0.0001:
            break
    return weight, bias   

In [13]:
def accuracy(weight,bias,data,y):
    m =data.shape[0]
    return sum((y*(np.dot(data,weight)+bias))>0)*100/m

In [14]:
lr = [1000,100,10,1,0.1,0.01,0.001]
best_lr=0
best_ac = 0
best_b = 0
best_w =0
for i in lr:
    w = np.array([0.1]*60).reshape((60,1))
    b=0.5
    w, b = logistic_regression("l0",1,w,b,i)
    ac = accuracy(w,b,valid_data,valid_y)
    if ac> best_ac:
        best_ac =ac
        best_lr =i
        best_b=b
        best_w = w

C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


In [15]:
best_ac

array([78.84615385])

In [16]:
best_lr

0.001

In [17]:
best_w,best_b

(array([[-38.88394747],
        [-27.76551832],
        [ 21.56638472],
        [-30.82776708],
        [ -4.34304187],
        [ 11.23869881],
        [  5.08054746],
        [ 56.0967234 ],
        [-53.31934313],
        [  3.96212492],
        [ -2.09450693],
        [ -4.40956303],
        [ -9.63626857],
        [  8.14318806],
        [ -9.65596739],
        [  6.40343128],
        [ 32.42485645],
        [-19.24313517],
        [ -6.96515723],
        [-10.34473506],
        [ 13.16877369],
        [-10.90342363],
        [  0.26208963],
        [ -5.34889249],
        [  0.92961055],
        [ -7.2613326 ],
        [ 15.38202779],
        [ -8.48375885],
        [  7.43346838],
        [-24.88302156],
        [ 34.96370145],
        [-16.38502983],
        [ -2.19013242],
        [ 22.48258175],
        [-23.66798381],
        [ 29.30201712],
        [ -7.64769664],
        [ -3.29452741],
        [  1.53696415],
        [ 16.21940288],
        [-27.58317321],
        [ 18.922

In [18]:
acc = accuracy(best_w,best_b,test_data,test_y)
print("Accuracy on Test set is:",acc)

Accuracy on Test set is: [76.92307692]


In [19]:
acc1 = accuracy(best_w,best_b,train_data,train_y)
print("Accuracy on train set is:",acc1)

Accuracy on train set is: [100.]


In [20]:
l = [0.001,0.01,0.1,1,10,100,1000]
lr = [0.001,0.01,0.1,1,10,100,1000]
Valid_acc_l2 = []
weight_list_l2=[]
bias_list_l2=[]
best_l_l2=0
best_acc_l2=0
best_weight_l2=0
best_bias_l2=0
best_lr_l2 = 0
for i in l:
    for j in lr:
        w = np.array([0.1]*60).reshape((60,1))
        b=0.5
        w, b = logistic_regression("l2",i,w,b,j)
        acc = accuracy(w,b,valid_data,valid_y)
        Valid_acc_l2.append(acc)
        weight_list_l2.append(w)
        bias_list_l2.append(b)
        if best_acc_l2<acc:
            best_acc_l2=acc
            best_weight_l2 =w
            best_bias_l2 = b
            best_l_l2=i
            best_lr_l2 = j

C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


In [21]:
best_acc_l2

array([80.76923077])

In [22]:
best_lr_l2, best_l_l2

(0.001, 0.001)

In [23]:
print (best_bias_l2, best_weight_l2)

[7.36378382] [[-19.66713623]
 [-12.75452138]
 [  9.16751103]
 [-16.96484394]
 [ -0.69719615]
 [  4.81009777]
 [  7.24423164]
 [ 28.67187088]
 [-28.70358962]
 [  0.4452169 ]
 [ -2.63391869]
 [ -3.84420733]
 [ -4.36935871]
 [  3.06598224]
 [ -5.73809783]
 [  4.2973959 ]
 [ 20.08873559]
 [-11.89948834]
 [ -3.98752848]
 [ -5.64621652]
 [  5.99999404]
 [ -5.63098692]
 [ -1.20111063]
 [ -1.3598877 ]
 [ -1.61299787]
 [  0.66076354]
 [  6.05789716]
 [ -4.04662904]
 [  3.83217514]
 [-13.03933713]
 [ 19.60865495]
 [-11.09819228]
 [ -0.251453  ]
 [ 11.73119625]
 [-11.58556087]
 [ 14.34656819]
 [ -2.24508768]
 [ -3.04445359]
 [  0.89858875]
 [ 11.17262427]
 [-14.20011012]
 [ 10.17240072]
 [-12.34050461]
 [ -3.59665276]
 [ -1.86718376]
 [  5.77086872]
 [-10.1324993 ]
 [-24.30251202]
 [-14.42216281]
 [ 10.46335923]
 [ -3.11981972]
 [ -5.59177041]
 [ -2.81896204]
 [ -3.35862269]
 [  1.81399392]
 [ -1.39309162]
 [  1.62388737]
 [ -5.19065944]
 [ -5.79376871]
 [ -5.13877949]]


In [24]:
Acc = accuracy(best_weight_l2,best_bias_l2,test_data,test_y)
print("Accuracy using l2 regularization with best weight and bias on Test set is:",Acc)

Accuracy using l2 regularization with best weight and bias on Test set is: [80.76923077]


In [25]:
accuracy(best_weight_l2,best_bias_l2,train_data,train_y)

array([99.03846154])

In [26]:
l = [1000,100,10,1,0.1,0.01,0.001]
lr = [1000,100,10,1,0.1,0.01,0.001]
Valid_acc_l1 = []
weight_list_l1=[]
bias_list_l1=[]
best_l_l1=0
best_acc_l1=0
best_weight_l1=0
best_bias_l1=0
best_lr_l1 = 0
for i in l:
    for j in lr:
        w = np.array([0.1]*60).reshape((60,1))
        b=0.5
        w, b = logistic_regression("l1",i,w,b,j)
        acc = accuracy(w,b,valid_data,valid_y)
        Valid_acc_l1.append(acc)
        weight_list_l1.append(w)
        bias_list_l1.append(b)
        if best_acc_l1<acc:
            best_acc_l1=acc
            best_weight_l1 =w
            best_bias_l1 = b
            best_l_l1=i
            best_lr_l1 =j

C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\miriy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


In [27]:
best_acc_l1

array([80.76923077])

In [28]:
best_lr_l1, best_l_l1

(0.001, 1)

In [29]:
print(best_bias_l1,best_weight_l1)

[1.08894541] [[-0.18191028]
 [-0.17553854]
 [-0.11932074]
 [-0.22730637]
 [-0.03616732]
 [ 0.09699172]
 [ 0.16238014]
 [ 0.00868702]
 [-0.79278901]
 [-0.84187603]
 [-1.11752658]
 [-1.06061883]
 [-0.80541676]
 [-0.22744702]
 [ 0.07501798]
 [ 0.52217558]
 [ 0.68421774]
 [ 0.28004727]
 [-0.35755668]
 [-0.64298641]
 [-0.34194892]
 [-0.25215069]
 [-0.38622567]
 [-0.15463364]
 [ 0.33892591]
 [ 0.58094714]
 [ 0.12162592]
 [-0.26326327]
 [-0.11956691]
 [-0.2121798 ]
 [ 0.19530047]
 [-0.45706834]
 [-0.0082746 ]
 [ 0.50647813]
 [ 0.54800413]
 [ 0.88301602]
 [ 0.61494623]
 [-0.1100696 ]
 [-0.12828989]
 [ 0.47496512]
 [ 0.20901707]
 [-0.14401016]
 [-0.83702146]
 [-0.86948864]
 [-0.76294462]
 [-0.51253826]
 [-0.47900509]
 [-0.48326593]
 [-0.27143062]
 [-0.01359121]
 [-0.09345307]
 [-0.07441654]
 [-0.028107  ]
 [-0.03665482]
 [ 0.01152312]
 [-0.02650072]
 [ 0.00722721]
 [-0.03860953]
 [-0.02709447]
 [-0.01855204]]


In [30]:
accuracy(best_weight_l1,best_bias_l1,train_data,train_y)

array([81.73076923])

In [31]:
Acc = accuracy(best_weight_l1,best_bias_l1,test_data,test_y)
print("Accuracy using l1 regularization with best weight and bias on Test set is:",Acc)

Accuracy using l1 regularization with best weight and bias on Test set is: [75.]
